<a href="https://colab.research.google.com/github/vhppacheco/ic-ita-finance-ia/blob/main/ibovespa_analysis/Ibovespa_Analysis_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 1 - Análise Preliminar Ibovespa

#### pip install

In [ ]:
!pip install -q plotly

In [ ]:
!pip install -q yfinance

In [ ]:
!pip install -q investpy

##### Instalação da Biblioteca Kora, que possibilita utilizar o Selenium de forma simplificada no Google Colab. [Link para acesso ao repositório](https://github.com/korakot/kora/blob/master/kora/selenium.py)

In [ ]:
!pip install -q kora

##Importanto as Bibliotecas

In [ ]:
from kora.selenium import wd
from time import sleep

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import yfinance as yf
import investpy

import os

import requests
from bs4 import BeautifulSoup

from google.colab import files
import warnings
warnings.filterwarnings('ignore')

## Webcrawling Carteira Ibovespa - Dia de Referência

In [ ]:
def carteira_teorica(indice):
  url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{}?language=pt-br'.format(indice.upper())
  wd.get(url)
  sleep(5)

  wd.find_element_by_id('segment').send_keys("Setor de Atuação")
  sleep(5)

  wd.find_element_by_link_text("Download").click()
  sleep(5)

  arquivo = !ls -1t *.csv
    
  return pd.read_csv(arquivo[0],sep=';',encoding='ISO-8859-1',engine='python',thousands='.', decimal=',', header= 1, index_col= False)

In [ ]:
ibov = carteira_teorica('ibov').iloc[:-2]

In [ ]:
#Obtendo do arquivo recém baixado e incorporando sua data de referência em uma coluna

arquivo = !ls -1t *.csv
arquivo_middle = arquivo[0].split('_')[1]
arquivo_date = arquivo_middle.split('.')[0]

ibov['Data Carteira'] = arquivo_date
ibov['Data Carteira'] = pd.to_datetime(ibov['Data Carteira'], errors='coerce')
ibov

In [ ]:
ibov.info()

In [ ]:
ibov.isna().sum()

In [ ]:
ativos_setor = ibov['Setor'].groupby(ibov['Setor']).count().sort_values(ascending=False)
ativos_setor = pd.DataFrame(ativos_setor).rename(columns={"Setor":"Nº de Ativos"})
ativos_setor = pd.merge(ativos_setor,ibov, how='left',on=['Setor']).drop_duplicates(subset=['Setor'])[['Setor','Nº de Ativos','Part. (%)Acum.']]
ativos_setor = ativos_setor.set_index('Setor')
ativos_setor

In [ ]:
y= ativos_setor['Nº de Ativos']
x = ativos_setor.index

fig = go.Figure(data=[go.Bar(x = x,
             y=y,
            text=y,
            textposition='auto',
            marker_color = 'blue'
        )])
fig.update_layout(height=500, width=1800,title_text='Nº de Ativos por Setor')
fig.show()

In [ ]:
y= ativos_setor['Part. (%)Acum.'].sort_values(ascending= False)
x = ativos_setor.index

fig = go.Figure(data=[go.Bar(x = x,
             y=y,
            text=y,
            textposition='auto',
            marker_color = 'red'
        )])
fig.update_layout(height=500, width=1800,title_text='Participação % por Setor')
fig.show()

## Webscraping Dados Fundamentalistas das Ações  do Ibovespa

##### O site utilizado como fonte é o Fundamentus. [Link do site](http://www.fundamentus.com.br/resultado.php)

In [ ]:
url = 'http://www.fundamentus.com.br/resultado.php?interface=mobile'

In [ ]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [ ]:
df_fund = pd.read_html(r.text,  decimal=',', thousands='.')[0]
df_fund = df_fund.rename(columns={"Papel":"Código"})

In [ ]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df_fund[coluna] = df_fund[coluna].str.replace('.', '')
  df_fund[coluna] = df_fund[coluna].str.replace(',', '.')
  df_fund[coluna] = df_fund[coluna].str.rstrip('%').astype('float') / 100

In [ ]:
df_fund

## Cruzando a Carteira do Ibovespa com os Indicadores Fundamentalistas

In [ ]:
ibov_fund = pd.merge(ibov,df_fund, how='left',on=['Código']).sort_values(by='P/L', ascending = True).reset_index().iloc[:,1:]
ibov_fund

In [ ]:
ibov_fund.isna().sum()

In [ ]:
ibov_fund.to_excel('Ibov+Fundamentos_17_02.xlsx')

In [ ]:
#Download na máquina local

arquivo = !ls -1t *.xlsx
files.download(arquivo[0])

#### Ler arquivo .xlsx caso precise

In [ ]:
ibov_fund = pd.read_excel('/content/Ibov+Fundamentos.xlsx').iloc[:,1:]
ibov_fund

## Obtenção dados CVM - Companhias Abertas

In [ ]:
r_cvm = requests.get('http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv')

rows = [i.strip().split(';') for i in r_cvm.text.split('\n')]
df_cvm = pd.DataFrame(rows[1:], columns=rows[0])[:-1]
df_cvm

In [ ]:
df_cvm.tail(3)

In [ ]:
df_cvm['DT_REG'] = pd.to_datetime(df_cvm['DT_REG'], errors='coerce')

In [ ]:
df_cvm.info()

## Cruzamento de bases Ibovespa+Fundamentos e CVM

### Webscraping Infomoney (busca por CNPJ como chave única entre as bases do IBOV+Fundamentos e CVM)

In [ ]:
url2 = 'https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-das-acoes-negociadas-em-bolsa-e-saiba-como-declarar-no-imposto-de-renda/'
response2 = requests.get(url2)

html2 = BeautifulSoup(response2.text, "html.parser")
html2.find_all('table')

In [ ]:
infomoney = pd.read_html(response2.text,header=0)[0]
infomoney

In [ ]:
infomoney.to_excel('cnpj_infomoney.xlsx')

#### Dados do Infomoney já ajustados (Nomes das Companhias)

In [ ]:
infomoney = pd.read_excel('cnpj_infomoney.xlsx').iloc[:,1:]
infomoney

In [ ]:
infomoney = infomoney.rename(columns={"Nome de pregão":"Ação"})

In [ ]:
ibov_fund_infomoney = pd.merge(ibov_fund, infomoney, how='left',on=['Ação']).reset_index().iloc[:,1:]
ibov_fund_infomoney

In [ ]:
ibov_fund_infomoney.isna().sum()

In [ ]:
filtro_nan= ibov_fund_infomoney[ibov_fund_infomoney['CNPJ'].isnull()]
filtro_nan

In [ ]:
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].str.replace('.', '')
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].str.replace('/', '')
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].str.replace('-', '')
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].astype(int)
ibov_fund_infomoney

#### Arrumando CNPJ Base CVM

In [ ]:
df_cvm = df_cvm.rename(columns={"CNPJ_CIA":"CNPJ"})
df_cvm['CNPJ'] = df_cvm['CNPJ'].str.replace('.', '')
df_cvm['CNPJ'] = df_cvm['CNPJ'].str.replace('/', '')
df_cvm['CNPJ'] = df_cvm['CNPJ'].str.replace('-','')
df_cvm['CNPJ'] = df_cvm['CNPJ'].astype(int)
df_cvm

In [ ]:
ibov_fund_cvm =  pd.merge(ibov_fund_infomoney,df_cvm, how='left',on=['CNPJ']).sort_values(by='P/L', ascending = True).reset_index().iloc[:,1:]
ibov_fund_cvm

In [ ]:
filtro_nan2= ibov_fund_cvm[ibov_fund_cvm['CNPJ'].isnull()]
filtro_nan2

In [ ]:
ibov_fund_cvm.columns

In [ ]:
ibov_fund_cvm = ibov_fund_cvm[['Setor', 'Código', 'Ação', 'Tipo', 'Qtde. Teórica', 'Part. (%)',
       'Part. (%)Acum.', 'Data Carteira', 'DT_REG', 'Cotação', 'P/L', 'P/VP', 'PSR',
       'Div.Yield', 'P/Ativo', 'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq',
       'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC',
       'ROE', 'Liq.2meses', 'Patrim. Líq', 'Dív.Líq/ Patrim.',
       'Cresc. Rec.5a','Dívida Líquida/EBITDA', 'Valor de Mercado']]

ibov_fund_cvm

In [ ]:
ibov_fund_cvm['DT_REG'] = pd.to_datetime(ibov_fund_cvm['DT_REG'], errors='coerce')
ibov_fund_cvm.info()

In [ ]:
ibov_fund_cvm = ibov_fund_cvm.rename(columns={"DT_REG":"Data Referência IPO"})
ibov_fund_cvm = ibov_fund_cvm.sort_values(by=['Código','Data Referência IPO']).drop_duplicates(subset=['Código'], keep='last').reset_index().iloc[:,1:]
ibov_fund_cvm.to_excel('Base_Ibov_final.xlsx')

In [ ]:
#Download na máquina local

arquivo = !ls -1t *.xlsx
files.download(arquivo[0])

In [ ]:
ibov_fund_cvm.iloc[:,1:]

In [ ]:
filtro_cod= ibov_fund_cvm[ibov_fund_cvm['Ação']=="LOCAWEB"]
filtro_cod

#### Selecionando ações com IPO antes de 2016

In [ ]:
base_ibov_pre = ibov_fund_cvm[(ibov_fund_cvm['Data Referência IPO'] <= '2016-01-01')] 
base_ibov_pre

In [ ]:
base_ibov_pre.to_excel('base_ibov_fundamentos_ipo_pre_2016.xlsx')

In [ ]:
#Download na máquina local

arquivo = !ls -1t *.xlsx
files.download(arquivo[0])

#### Selecionando ações com IPO pós de 2016



In [ ]:
base_ibov_pos = ibov_fund_cvm[(ibov_fund_cvm['Data Referência IPO'] >= '2016-01-01')] 
base_ibov_pos

In [ ]:
base_ibov_pos[['Código','Data Referência IPO']].reset_index().iloc[:,1:]

## Seleção da cesta de ações

In [ ]:
base_ibov_pre.sort_values(by='ROE', ascending = True)

In [ ]:
all_data = []  

for i in range(10,29):
  base2 = base_ibov_pre.iloc[:,i].groupby(base_ibov_pre['Setor']).mean().sort_values(ascending=False)
  base2 = pd.DataFrame(base2)
  all_data.append(base2)

base= pd.concat(all_data, axis=1)
base

In [ ]:
fig = make_subplots(rows=2, cols=3)
 
fig.append_trace(go.Bar(name = "Média P/L",
    y=base['P/L'],
    x=base.index,
), row=1, col=1)
 
fig.append_trace(go.Bar(name = "Média P/VP",
    y=base['P/VP'],
    x=base.index,
), row=1, col=2)
 
fig.append_trace(go.Bar(name = "Média Div.Yield",
    y=round(base['Div.Yield']*100,2),
    x=base.index,
), row=1, col=3)

fig.append_trace(go.Bar(name = "Média EV/EBITDA",
    y=base['EV/EBITDA'],
    x=base.index,
), row=2, col=1)
 
fig.append_trace(go.Bar(name = "Patrim. Líq",
    y=base['Patrim. Líq'],
    x=base.index,
), row=2, col=2)
 
fig.append_trace(go.Bar(name = "Média ROE",
    y=round(base['ROE']*100,2),
    x=base.index,
), row=2, col=3)
 
 
fig.update_layout(height=1400,width=1900,title_text="Médias Múltiplos por Setor",font_size=7.5)
fig.update_xaxes(rangeselector_font_size=5)
fig.show()

#### Função criada para fornecer agilidade nas análises, conforme abaixo:

*  Dividiu-se entre múltiplos "maior melhor" e "menor melhor". Dessa forma, a escolha dos ativos passou por uma ponderada simples que relaciona os múltiplos da seguinte forma através de um Classificador: 

        Classificador = (1/múltiplo "menor melhor")*(múltiplo "maior melhor")


* Na análise abaixo, portanto, utilizou-se  múltiplo P/L como "menor melhor" e o ROE como "Maior Melhor", selecionando-se 15 empresas com o Classificador gerado em ordem crescente (0). Outra opção seria colocar o Classicador em ordem descrescente (1)

#### Múltiplos possíveis a serem analisados:
'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a','Dívida Líquida/EBITDA', 'Valor de Mercado'





In [ ]:
def criterio_carteira(multiplo_maior_melhor,multiplo_menor_melhor,qtd,superior_inferior):

  base_new = base_ibov_pre

  for i in multiplo_menor_melhor:
    base_new[i] = (1/base_new[i])

  sum_list = multiplo_maior_melhor + multiplo_menor_melhor

  base_new['Classificador'] = 1

  for n in sum_list:
    base_new['Classificador'] =  base_new['Classificador']*base_new[n] 

  if superior_inferior == 0:
    base_new = base_new.sort_values(by = ['Classificador'],ascending = True)
    return base_new['Código'].iloc[:qtd]

  elif superior_inferior == 1:
    base_new = base_new.sort_values(by = ['Classificador'],ascending = False)
    return base_new['Código'].iloc[:qtd]

  else:
    print("Você tem que escolher 0 ou 1")

In [ ]:
multiplo_maior_melhor = ['ROE']
multiplo_menor_melhor = ['P/L']
qtd_ativos = 15
ordem = 0

ativos = criterio_carteira(multiplo_maior_melhor,multiplo_menor_melhor,qtd_ativos,ordem)
ativos = list(ativos)
ativos

#### Função para inserir S.A nos tickers

In [ ]:
sa = '.SA'

ativos_sa = []

for item in ativos:
    ativos_sa.append(str(item)+sa)

ativos_sa= list(ativos_sa)
ativos_sa

## Obtendo dados via *Yahoo Finance*

In [ ]:
df= pd.DataFrame()

inicio = '2016-01-01'
fim = '2020-12-31'

for ativo in ativos_sa:
  df[ativo] = yf.download(ativo, start = inicio, end = fim)['Adj Close']

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df.plot(figsize = (20,10))

In [ ]:
preco_normalizado = df/df.iloc[0]
preco_normalizado

In [ ]:
preco_normalizado.plot(figsize = (20,10))

### Log Retornos

In [ ]:
df_log = (df.pct_change(axis=0)).apply(np.log1p)
df_log= df_log.iloc[1:,:]
df_log

In [ ]:
plt.rcParams.update({'font.size': 15})
plt.rc('legend', fontsize=15) 
df_log.plot(figsize=(30,17))
plt.ylabel('Retornos Logarítmicos')
plt.xlabel('Data')
plt.show()

In [ ]:
df_log.plot.hist(bins=100,figsize=(20,17));

In [ ]:
correlacao = df_log.corr()
correlacao

In [ ]:
plt.figure(figsize = (30, 20))
sns.heatmap(round(correlacao,2), annot = True, cmap = "Blues")

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

plt.figure(figsize=(18,7))
distances = np.sqrt((1 - correlacao) / 2)
clusters = linkage(squareform(distances), method="ward")

dendrogram(clusters, labels=df_log.columns);

###Risco x Retorno

In [ ]:
df_log.std()

In [ ]:
volatilidade = pd.DataFrame(df_log.std(), columns=['Vol'])

retornos_log_medios = pd.DataFrame(df_log.mean(), columns=['Retornos'])

In [ ]:
volatilidade

In [ ]:
y = round(volatilidade['Vol'],4)
x = volatilidade.index

color = np.where(y<0, 'red', 'blue')

fig = go.Figure(data=[go.Bar(x = x,
             y=y,
            text=y,
            textposition='auto',
            marker_color = color
        )])
fig.update_layout(height=525, width=1000,title_text='Volatilidade por Ação')
fig.show()

In [ ]:
retornos_log_medios

In [ ]:
y = round(retornos_log_medios['Retornos'],4)
x = retornos_log_medios.index

color = np.where(y<0, 'red', 'blue')

fig = go.Figure(data=[go.Bar(x = x,
             y=y,
            text=y,
            textposition='auto',
            marker_color = color
        )])
fig.update_layout(height=525, width=1000,title_text='Log Retorno Médio por Ação')
fig.show()

In [ ]:
risco_retorno = pd.concat([retornos_log_medios, volatilidade], axis = 1)
risco_retorno

### Plotando

In [ ]:
plt.subplots(figsize = (10,8))

sns.scatterplot(data = risco_retorno, x = 'Vol', y = 'Retornos')

for i in range(risco_retorno.shape[0]):
  plt.text(x = risco_retorno.Vol[i] , y = risco_retorno.Retornos[i], s = risco_retorno.index[i],
           fontdict = dict(color = 'white', size = 15),
           bbox = dict(facecolor = 'Blue'))